In [1]:
from pandas import DataFrame, MultiIndex, merge, read_csv, concat
import pandas as pd


from utils.data import get_dataframe, remove_warmup_df
from utils.global_config import (
    N_WARMUPS,
    SERIES_COLUMN_NAME,
    TIMESTEPS_COLUMN_NAME,
    DATA_DIR,
    TEST_FILE,
)
from utils.errors_computing import compute_errors_df

from utils.errors_computing_train import compute_errors_df_omg_so_ugly

# Checking the post-processing

In [2]:
get_dataframe(DATA_DIR + "/" + TEST_FILE)

,individus,temps,x1,x1_obs,x2,x2_obs,x3,x3_obs,x4,x4_obs,...,x7,x7_obs,x8,x2_x5,x4_x7,x6_x8,y_mixed,y_mixed_obs,y_fixed,y_fixed_obs
0,1,0,7.642962,6.541205,-4.901404,-4.930665,-2.762202,-2.708554,-0.524125,-0.684053,...,4.834070,3.941196,0,-18.623302,-2.533655,0.000000,-14.147090,-16.401119,-10.270811,-10.709328
1,1,1,9.115644,9.473100,-4.442349,-4.361045,-2.761245,-2.881917,-0.412815,-0.441288,...,2.628120,2.634557,0,-17.325424,-1.084928,0.000000,-13.233794,-13.504805,-9.577035,-9.098006
2,1,2,10.588327,9.902374,-4.173819,-4.263328,-2.758375,-2.779620,-0.312098,-0.283281,...,1.182562,0.864215,0,-16.696917,-0.369076,0.000000,-12.791824,-12.984600,-9.241134,-9.166685
3,1,3,12.061009,12.529263,-3.983294,-3.820831,-2.753592,-2.649039,-0.220966,-0.209213,...,0.478113,0.074657,0,-16.333284,-0.105647,0.000000,-12.532933,-12.428293,-9.046100,-7.888457
4,1,4,13.533692,13.084103,-3.835512,-3.638428,-2.746894,-2.827505,-0.138506,-0.011992,...,0.184193,-0.450209,0,-16.109461,-0.025512,0.000000,-12.371850,-13.117012,-8.925675,-9.075263
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12995,500,21,53.020481,52.321472,-2.272128,-2.236098,-1.706892,-1.640332,1.684388,1.799199,...,0.228313,0.257998,1,-20.101305,0.384568,33.766777,14.255805,13.533262,3.614660,3.089746
12996,500,22,55.208542,55.218037,-2.240013,-2.426036,-1.688810,-1.665168,1.700566,1.514891,...,0.193649,-0.115100,1,-19.826836,0.329314,34.252884,14.365780,14.035857,3.974612,1.382215
12997,500,23,57.396603,56.736217,-2.209264,-2.319933,-1.669887,-1.666301,1.715204,1.755337,...,0.164140,0.788623,1,-19.561745,0.281534,34.742466,14.480370,15.106890,4.330982,4.661281
12998,500,24,59.584660,59.882187,-2.179771,-2.047545,-1.650123,-1.888849,1.728449,1.740999,...,0.139049,-0.247184,1,-19.305777,0.240339,35.235523,14.599297,12.746320,4.683908,3.172152


In [3]:
df_tgt = get_dataframe(DATA_DIR + "/" + TEST_FILE)
df_tgt = df_tgt[
    [SERIES_COLUMN_NAME, TIMESTEPS_COLUMN_NAME] + ["y_fixed", "y_fixed_obs"]
]
df_tgt = remove_warmup_df(df_tgt, N_WARMUPS)
df_tgt = df_tgt.set_index([SERIES_COLUMN_NAME, TIMESTEPS_COLUMN_NAME])
df_tgt

y_fixed  y_fixed_obs
individus temps                       
1         5     -8.850711    -8.181961
          6     -8.805703    -9.169623
          7     -8.781418    -9.239458
          8     -8.771918    -9.850792
          9     -8.773167    -7.914300
...                   ...          ...
500       21     3.614660     3.089746
          22     3.974612     1.382215
          23     4.330982     4.661281
          24     4.683908     3.172152
          25     5.033576     5.850792

[10500 rows x 2 columns]

In [4]:
study = "SF_FE"

df = read_csv(study + "/predictions.csv")
df = remove_warmup_df(df, N_WARMUPS)
df = df.set_index(["simulation", "iseed", SERIES_COLUMN_NAME, TIMESTEPS_COLUMN_NAME])
df

pred
simulation       iseed individus temps          
simulation71.csv 0     1         5     -8.816993
                                 6     -8.764876
                                 7     -8.735535
                                 8     -8.722729
                                 9     -8.721816
...                                          ...
simulation50.csv 4     500       21     3.674401
                                 22     4.038134
                                 23     4.398133
                                 24     4.754548
                                 25     5.107569

[5250000 rows x 1 columns]

In [5]:
assert set(df.index.get_level_values("temps").unique()) == set(
    df_tgt.index.get_level_values("temps").unique()
)
assert set(df.index.get_level_values("individus").unique()) == set(
    df_tgt.index.get_level_values("individus").unique()
)

In [6]:
# merging prediction and targets
df_mrg = merge(left=df, right=df_tgt, on=["individus", "temps"], how="left")
df_mrg.index = df.index
df_mrg

pred   y_fixed  y_fixed_obs
simulation       iseed individus temps                                 
simulation71.csv 0     1         5     -8.816993 -8.850711    -8.181961
                                 6     -8.764876 -8.805703    -9.169623
                                 7     -8.735535 -8.781418    -9.239458
                                 8     -8.722729 -8.771918    -9.850792
                                 9     -8.721816 -8.773167    -7.914300
...                                          ...       ...          ...
simulation50.csv 4     500       21     3.674401  3.614660     3.089746
                                 22     4.038134  3.974612     1.382215
                                 23     4.398133  4.330982     4.661281
                                 24     4.754548  4.683908     3.172152
                                 25     5.107569  5.033576     5.850792

[5250000 rows x 3 columns]

In [7]:
tgt_cols = [c for c in df_mrg.columns if c.startswith("y_")]

for col in tgt_cols:
    print(col)
    # petite vérif du merge… normalement les targets sont dupliquées pour chaque simulation
    assert all(df_mrg[[col]].groupby(["iseed", "individus", "temps"]).std() == 0)
    #
    df_mrg["error_" + col] = df_mrg[col] - df_mrg["pred"]

df_mrg

y_fixed
y_fixed_obs


pred   y_fixed  y_fixed_obs  \
simulation       iseed individus temps                                    
simulation71.csv 0     1         5     -8.816993 -8.850711    -8.181961   
                                 6     -8.764876 -8.805703    -9.169623   
                                 7     -8.735535 -8.781418    -9.239458   
                                 8     -8.722729 -8.771918    -9.850792   
                                 9     -8.721816 -8.773167    -7.914300   
...                                          ...       ...          ...   
simulation50.csv 4     500       21     3.674401  3.614660     3.089746   
                                 22     4.038134  3.974612     1.382215   
                                 23     4.398133  4.330982     4.661281   
                                 24     4.754548  4.683908     3.172152   
                                 25     5.107569  5.033576     5.850792   

                                        error_y_fixed  error_y_fixed_obs  
simulation       iseed individus temps                                    
simulation71.csv 0     1         5          -0.033718           0.635032  
                                 6          -0.040827          -0.404747  
                                 7          -0.045883          -0.503923  
                                 8          -0.049189          -1.128062  
                                 9          -0.051350           0.807516  
...                                               ...                ...  
simulation50.csv 4     500       21         -0.059742          -0.584656  
                                 22         -0.063522          -2.655919  
                                 23         -0.067152           0.263147  
                                 24         -0.070640          -1.582396  
                                 25         -0.073993           0.743223  

[5250000 rows x 5 columns]

In [8]:
err_cols = [c for c in df_mrg.columns if c.startswith("error_")]

df_dict = {}

# we have the same seed number for each indiv/tsep, so  mean(mean) == mean

for col in err_cols:
    df_dict[("mae", col)] = df_mrg[col].abs().groupby("simulation").mean().mean()

for col in err_cols:
    df_dict[("mse", col)] = df_mrg[col].pow(2).groupby("simulation").mean().mean()

In [9]:
df_res = DataFrame(
    [df_dict], columns=MultiIndex.from_tuples(df_dict.keys()), index=[study]
)
df_res

mae                             mse                  
      error_y_fixed error_y_fixed_obs error_y_fixed error_y_fixed_obs
SF_FE      0.042008          0.793757      0.004236            0.9894

# Mean errors for each studies

pred_measure = meand(pred_seed)  
SE_measure = (target_measure - pred_measure)  
MSE_simulation = mean(nowarmup(SE_measure))  
MSE = mean(MSE_simulation)  




In [10]:
import SF_FE.study_config
import SF_ME.study_config
import AF_FE.study_config
import AF_ME.study_config

## Prediction on test set

In [11]:
df_err_test = DataFrame()
df_err_test = concat(
    [
        compute_errors_df(SF_FE.study_config),
        compute_errors_df(SF_ME.study_config),
        compute_errors_df(AF_FE.study_config),
        compute_errors_df(AF_ME.study_config),
    ]
)

In [12]:
pd.options.display.float_format = "{:.3f}".format
df_err_test

y_fixed       y_fixed_obs       y_mixed         y_mixed_obs        
          mae   mse         mae   mse     mae     mse         mae     mse
SF_FE   0.042 0.004       0.794 0.989     NaN     NaN         NaN     NaN
SF_ME     NaN   NaN         NaN   NaN   9.688 254.368       9.792 255.218
AF_FE   0.258 0.154       0.846 1.137     NaN     NaN         NaN     NaN
AF_ME     NaN   NaN         NaN   NaN  10.813 276.199      10.869 277.134

## Prediction on train set

In [13]:
df_err_train = DataFrame()
df_err_train = concat(
    [
        compute_errors_df_omg_so_ugly(SF_FE.study_config),
        compute_errors_df_omg_so_ugly(SF_ME.study_config),
        compute_errors_df_omg_so_ugly(AF_FE.study_config),
        compute_errors_df_omg_so_ugly(AF_ME.study_config),
    ]
)

In [14]:
pd.options.display.float_format = "{:.3f}".format
df_err_train

y_fixed       y_fixed_obs       y_mixed         y_mixed_obs        
          mae   mse         mae   mse     mae     mse         mae     mse
SF_FE   0.045 0.006       0.800 1.005     NaN     NaN         NaN     NaN
SF_ME     NaN   NaN         NaN   NaN  10.397 288.699      10.502 289.703
AF_FE   0.183 0.064       0.807 1.024     NaN     NaN         NaN     NaN
AF_ME     NaN   NaN         NaN   NaN  11.338 299.317      11.385 300.317